In [177]:
import numpy as np
import matplotlib as plt
import cv2
import os

frame Position of the sample within the sequence

track id Tracking ID of the object within the sequence

type Object type: ’Car’, ’Pedestrian’, ’Cyclist’, ’Tram’, ’Person_sitting’, ’Misc’ or ’DontCare’

truncated Integer (0,1,2) indicating the level of truncation.

occluded Integer (0,1,2,3) indicating occlusion state.

alpha Observation angle of object, ranging [-Pi; Pi]

bbox 2D (0-based) bounding box of the object: Left, top, right, bottom image coordinates

dimensions 3D object dimensions: height, width, length [m]

location 3D object location x,y,z in camera coords. [m]

rotation_y Rotation around Y-axis in camera coords. [-Pi; Pi]

In [178]:
# i assume you extracted tracking dataset here
main_folder = "/home/essys/datasets/kitty/tracking/"
# and labels here
labels_folder = main_folder + "labels/training/label_02/"
label_files = os.listdir(labels_folder)

In [179]:
img_folder = main_folder + "data_tracking_image_2/training/image_02/"
img_folders = os.listdir(img_folder)

In [180]:
detections = {}
for i in range(len(label_files)):
    with open(labels_folder + label_files[i], 'r') as f:
        dets = f.read().split('\n')
        print(np.size(dets))
        detections[label_files[i]] = dets


355
2149
4360
11193
1795
1447
3496
2013
4272
3723
799
2089
5302
1324
2099
1500
1090
2411
8897
862
4146


In [181]:
label = detections[label_files[0]][0].split(' ')
frame_id = label[0]
track_id = label[1]
track_type = label[2]#3,4,5,
#Left, top, right, bottom
bbox = [label[6],label[7], label[8],label[9]]

In [184]:
types = []
max_track = 200
for img_folder_name, label_file_name in zip(np.sort(img_folders),np.sort(label_files) ):
    
    #getting image file names
    img_files = np.sort(os.listdir(img_folder + img_folder_name))
    # getting the detections for that files
    dets = detections[label_file_name]
    tracks = {}
    # and loop for each frame
    for img_file in img_files:
        # getting frame id
        frame_id = int(img_file.split('.png')[0])
        frame = cv2.imread(img_folder + img_folder_name +"/" +img_file, -1)
        # for that frame_id find all car bboxes
        for det in dets:
            if(len(det) <= 0):break
            label = det.split(' ')
            frame_id_det = int(label[0])
            if(not frame_id  == frame_id_det): continue
            track_id = label[1]
            track_type = label[2]#3,4,5,
#             if(not track_type == 'Cyclist'):continue
            types.append(track_type)
            #Left, top, right, bottom
            bbox = np.array([label[6],label[7], label[8],label[9]], dtype=np.float32)
#             print( track_id, track_type, bbox)
            

            # represents the top left corner of rectangle 
            start_point = (bbox[0], bbox[1]) 

            # represents the bottom right corner of rectangle 
            end_point = (bbox[2], bbox[3]) 

            # Blue color in BGR 
            color = (255, 0, 0) 

            # Line thickness of 2 px 
            thickness = 2
            crop = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2]) ]
#             print(np.shape(crop), int(bbox[0]), int(bbox[2]), int(bbox[1]),int(bbox[3]), np.shape(frame))
#             frame = cv2.rectangle(frame, start_point, end_point, color, thickness) 
            
            if(not track_id in tracks):
                tracks[track_id] = {"type":track_type, "crops":{"count":1,"1":crop}, "id":track_id}
            else:
#                 print(tracks[track_id]["crops"]["count"], tracks[track_id]["id"])
                count = int(tracks[track_id]["crops"]["count"]) + 1
                tracks[track_id]["crops"]["count"] = str(count)
                tracks[track_id]["crops"][str(count)] = crop
            print(track_id, tracks[track_id]["crops"]["count"])
                
    # saving tracks
    keys = list(tracks.keys())
    temp_max_track = max_track
    for i in range(len(keys)):
        # there is only possible negative key is -1 so we can use it like:
        if(not keys[i] > '-1'): continue
        # saving by their type
        id_track = str(max_track+int(keys[i]) + 1)
        if(int(id_track) > temp_max_track): temp_max_track = int(id_track)
        id_track = "0"*(4 - len(id_track)) + id_track
#         folder_name = main_folder + "tracklets/" +  tracks[keys[i]]["type"] +"/"+ id_track + "/"
        folder_name = main_folder + "bbox_train/" +  id_track + "/"
        os.makedirs(folder_name,  exist_ok=True)
        
        crops = tracks[keys[i]]["crops"]
        count = int(tracks[keys[i]]["crops"]["count"])
#         print(list(crops.keys()), count)
        for j in range(count):
            id_img = str(j+1)
            id_img = "0"*(4 - len(id_img)) + id_img
            img_name = id_track + "C1T0001F" +id_img + ".jpg"
            crop = crops[str(j+1)]
            crop = cv2.resize(crop, (128, 256))
            cv2.imwrite(folder_name + img_name, crop)
    max_track = temp_max_track
#     break    
    
#         cv2.imshow("window_name", tracks["1"]["crops"][-1]) 
#         if(cv2.waitKey(100) == 27): break;
        
# cv2.destroyAllWindows()

-1 1
-1 2
0 1
1 1
2 1
-1 3
-1 4
0 2
1 2
2 2
-1 5
-1 6
0 3
1 3
2 3
-1 7
-1 8
0 4
1 4
2 4
-1 9
-1 10
0 5
1 5
2 5
-1 11
-1 12
0 6
1 6
2 6
3 1
-1 13
-1 14
0 7
1 7
3 2
-1 15
0 8
1 8
3 3
-1 16
0 9
1 9
3 4
-1 17
0 10
1 10
3 5
-1 18
0 11
1 11
3 6
-1 19
0 12
1 12
3 7
-1 20
0 13
1 13
3 8
-1 21
0 14
1 14
3 9
-1 22
0 15
1 15
3 10
-1 23
0 16
1 16
3 11
-1 24
0 17
1 17
3 12
-1 25
0 18
1 18
3 13
-1 26
0 19
1 19
3 14
-1 27
0 20
1 20
3 15
-1 28
0 21
1 21
3 16
-1 29
0 22
1 22
3 17
-1 30
0 23
1 23
3 18
-1 31
-1 32
0 24
1 24
3 19
-1 33
-1 34
0 25
1 25
3 20
-1 35
-1 36
0 26
1 26
3 21
-1 37
-1 38
0 27
1 27
3 22
-1 39
-1 40
0 28
1 28
3 23
-1 41
-1 42
0 29
1 29
3 24
-1 43
-1 44
0 30
1 30
3 25
-1 45
-1 46
0 31
1 31
3 26
-1 47
-1 48
0 32
1 32
3 27
-1 49
-1 50
0 33
1 33
3 28
-1 51
-1 52
0 34
1 34
3 29
-1 53
-1 54
0 35
1 35
3 30
-1 55
-1 56
0 36
1 36
3 31
-1 57
-1 58
0 37
1 37
3 32
-1 59
-1 60
0 38
1 38
3 33
-1 61
-1 62
0 39
1 39
3 34
-1 63
-1 64
0 40
1 40
3 35
-1 65
0 41
1 41
3 36
-1 66
0 42
1 42
3 37
-1 67
0 43


-1 224
-1 225
-1 226
-1 227
-1 228
4 33
5 33
6 33
7 19
9 14
11 3
-1 229
-1 230
-1 231
-1 232
-1 233
-1 234
4 34
5 34
6 34
7 20
9 15
11 4
-1 235
-1 236
-1 237
-1 238
-1 239
-1 240
4 35
5 35
6 35
7 21
9 16
10 1
11 5
13 1
-1 241
-1 242
-1 243
-1 244
-1 245
-1 246
4 36
5 36
6 36
7 22
8 1
9 17
10 2
11 6
13 2
-1 247
-1 248
-1 249
-1 250
-1 251
-1 252
4 37
5 37
6 37
7 23
8 2
9 18
10 3
11 7
13 3
-1 253
-1 254
-1 255
-1 256
-1 257
-1 258
4 38
5 38
6 38
7 24
8 3
9 19
10 4
11 8
13 4
-1 259
-1 260
-1 261
-1 262
-1 263
-1 264
-1 265
4 39
5 39
6 39
7 25
8 4
9 20
10 5
11 9
13 5
-1 266
-1 267
-1 268
-1 269
-1 270
-1 271
5 40
6 40
7 26
8 5
9 21
10 6
11 10
13 6
89 1
-1 272
-1 273
-1 274
-1 275
-1 276
-1 277
5 41
6 41
7 27
8 6
9 22
10 7
11 11
13 7
89 2
-1 278
-1 279
-1 280
-1 281
-1 282
-1 283
5 42
6 42
7 28
8 7
9 23
10 8
11 12
13 8
89 3
-1 284
-1 285
-1 286
-1 287
-1 288
-1 289
5 43
6 43
7 29
8 8
9 24
10 9
11 13
13 9
89 4
-1 290
-1 291
-1 292
-1 293
-1 294
-1 295
-1 296
5 44
6 44
7 30
8 9
9 25
10 10
11 

-1 657
-1 658
-1 659
-1 660
37 25
38 20
39 23
40 21
41 21
42 11
43 10
-1 661
-1 662
-1 663
37 26
38 21
39 24
40 22
41 22
42 12
43 11
44 1
45 1
-1 664
-1 665
-1 666
37 27
38 22
39 25
40 23
41 23
42 13
43 12
44 2
45 2
-1 667
-1 668
-1 669
38 23
39 26
40 24
41 24
42 14
43 13
44 3
45 3
-1 670
-1 671
-1 672
38 24
39 27
40 25
41 25
42 15
43 14
44 4
45 4
-1 673
-1 674
-1 675
-1 676
38 25
39 28
40 26
41 26
42 16
43 15
44 5
45 5
-1 677
-1 678
-1 679
39 29
40 27
41 27
42 17
43 16
44 6
45 6
46 1
-1 680
-1 681
-1 682
39 30
40 28
41 28
42 18
43 17
44 7
45 7
46 2
-1 683
-1 684
-1 685
39 31
40 29
41 29
42 19
43 18
44 8
45 8
46 3
-1 686
-1 687
-1 688
40 30
41 30
42 20
43 19
44 9
45 9
46 4
-1 689
-1 690
-1 691
40 31
41 31
42 21
43 20
44 10
45 10
46 5
-1 692
-1 693
-1 694
40 32
41 32
42 22
43 21
44 11
45 11
46 6
90 1
-1 695
-1 696
-1 697
40 33
41 33
42 23
43 22
44 12
45 12
46 7
90 2
-1 698
-1 699
-1 700
40 34
41 34
42 24
43 23
44 13
45 13
46 8
90 3
-1 701
-1 702
-1 703
40 35
41 35
42 25
43 24
44 14
45 1

-1 1041
-1 1042
-1 1043
-1 1044
63 76
67 23
68 13
69 13
70 13
72 8
75 22
-1 1045
-1 1046
-1 1047
-1 1048
63 77
67 24
68 14
69 14
70 14
72 9
75 23
-1 1049
-1 1050
-1 1051
-1 1052
-1 1053
63 78
67 25
68 15
69 15
70 15
72 10
75 24
-1 1054
-1 1055
-1 1056
63 79
67 26
68 16
69 16
70 16
71 1
72 11
73 1
74 1
75 25
-1 1057
-1 1058
-1 1059
67 27
68 17
69 17
70 17
71 2
72 12
73 2
74 2
75 26
-1 1060
-1 1061
-1 1062
67 28
68 18
69 18
70 18
71 3
72 13
73 3
74 3
75 27
-1 1063
-1 1064
-1 1065
67 29
68 19
69 19
70 19
71 4
72 14
73 4
74 4
75 28
-1 1066
-1 1067
-1 1068
67 30
68 20
69 20
70 20
71 5
72 15
73 5
74 5
75 29
-1 1069
-1 1070
-1 1071
67 31
68 21
69 21
70 21
71 6
72 16
73 6
74 6
75 30
-1 1072
-1 1073
-1 1074
67 32
68 22
69 22
70 22
71 7
72 17
73 7
74 7
75 31
-1 1075
-1 1076
-1 1077
67 33
68 23
69 23
70 23
71 8
72 18
73 8
74 8
75 32
-1 1078
-1 1079
-1 1080
67 34
68 24
69 24
70 24
71 9
72 19
73 9
74 9
75 33
-1 1081
-1 1082
-1 1083
67 35
69 25
70 25
71 10
72 20
73 10
74 10
75 34
-1 1084
-1 1085
-1 

-1 278
-1 279
-1 280
-1 281
1 48
2 15
3 20
4 1
5 21
7 11
-1 282
-1 283
-1 284
-1 285
1 49
2 16
3 21
4 2
5 22
7 12
-1 286
-1 287
-1 288
-1 289
1 50
2 17
3 22
4 3
5 23
7 13
-1 290
-1 291
-1 292
-1 293
1 51
2 18
3 23
4 4
5 24
7 14
-1 294
-1 295
-1 296
-1 297
1 52
2 19
3 24
4 5
5 25
7 15
-1 298
-1 299
-1 300
1 53
2 20
3 25
4 6
5 26
7 16
-1 301
-1 302
1 54
2 21
3 26
4 7
5 27
7 17
14 1
-1 303
-1 304
1 55
2 22
3 27
4 8
5 28
7 18
14 2
-1 305
-1 306
1 56
2 23
3 28
4 9
5 29
7 19
14 3
-1 307
-1 308
1 57
2 24
3 29
4 10
5 30
7 20
14 4
-1 309
-1 310
1 58
2 25
3 30
4 11
5 31
7 21
14 5
16 1
17 1
-1 311
-1 312
1 59
2 26
3 31
4 12
5 32
7 22
14 6
16 2
17 2
-1 313
-1 314
1 60
2 27
3 32
4 13
5 33
6 1
7 23
14 7
16 3
17 3
-1 315
-1 316
1 61
2 28
3 33
4 14
5 34
6 2
7 24
14 8
16 4
17 4
-1 317
-1 318
-1 319
1 62
2 29
3 34
4 15
5 35
6 3
7 25
14 9
16 5
17 5
-1 320
-1 321
-1 322
1 63
2 30
3 35
4 16
5 36
6 4
7 26
14 10
16 6
17 6
-1 323
-1 324
1 64
2 31
3 36
4 17
5 37
6 5
7 27
14 11
15 1
16 7
17 7
-1 325
-1 326
1 65

...

-1 529
-1 530
12 128
30 56
31 14
33 18
35 15
38 83
-1 531
-1 532
12 129
30 57
31 15
33 19
35 16
38 84
-1 533
-1 534
12 130
30 58
31 16
38 85
-1 535
-1 536
12 131
30 59
31 17
38 86
-1 537
-1 538
12 132
30 60
31 18
37 1
38 87
-1 539
-1 540
12 133
30 61
31 19
37 2
38 88
-1 541
-1 542
12 134
30 62
31 20
37 3
38 89
39 1
40 1
-1 543
-1 544
12 135
30 63
31 21
37 4
38 90
39 2
40 2
-1 545
-1 546
12 136
30 64
31 22
37 5
38 91
39 3
40 3
-1 547
12 137
30 65
31 23
37 6
38 92
39 4
40 4
-1 548
12 138
30 66
31 24
37 7
38 93
39 5
40 5
-1 549
12 139
30 67
31 25
37 8
38 94
39 6
40 6
-1 550
12 140
30 68
31 26
34 1
37 9
38 95
39 7
40 7
-1 551
12 141
31 27
34 2
37 10
38 96
39 8
40 8
-1 552
12 142
31 28
34 3
37 11
38 97
39 9
40 9
-1 553
12 143
31 29
34 4
37 12
38 98
39 10
40 10
-1 554
12 144
31 30
34 5
37 13
38 99
39 11
40 11
-1 555
12 145
31 31
34 6
37 14
38 100
39 12
40 12
-1 556
12 146
31 32
34 7
37 15
38 101
39 13
40 13
-1 557
12 147
31 33
34 8
37 16
38 102
39 14
40 14
-1 558
12 148
31 34
34 9
38 103
39 

-1 886
-1 887
-1 888
12 267
52 61
53 30
54 9
55 61
56 21
58 6
61 49
104 6
105 8
106 2
-1 889
-1 890
12 268
52 62
53 31
54 10
55 62
56 22
58 7
61 50
104 7
105 9
106 3
-1 891
-1 892
12 269
52 63
54 11
55 63
56 23
58 8
61 51
104 8
105 10
106 4
-1 893
-1 894
12 270
52 64
54 12
55 64
56 24
58 9
61 52
104 9
105 11
106 5
-1 895
-1 896
12 271
54 13
55 65
56 25
58 10
61 53
104 10
105 12
106 6
-1 897
-1 898
12 272
54 14
55 66
56 26
58 11
61 54
105 13
106 7
-1 899
-1 900
12 273
54 15
55 67
56 27
58 12
61 55
105 14
106 8
-1 901
-1 902
12 274
54 16
55 68
56 28
58 13
61 56
105 15
106 9
-1 903
-1 904
12 275
54 17
55 69
56 29
58 14
61 57
105 16
106 10
-1 905
-1 906
-1 907
12 276
54 18
55 70
56 30
57 1
58 15
61 58
65 1
105 17
106 11
-1 908
-1 909
-1 910
12 277
54 19
55 71
56 31
57 2
58 16
61 59
65 2
105 18
106 12
-1 911
-1 912
-1 913
12 278
54 20
55 72
56 32
57 3
58 17
61 60
65 3
105 19
106 13
-1 914
-1 915
12 279
54 21
55 73
56 33
57 4
58 18
61 61
65 4
105 20
106 14
107 1
-1 916
-1 917
12 280
54 22
55

-1 1206
-1 1207
-1 1208
-1 1209
12 375
81 39
82 5
83 52
84 4
86 25
88 4
93 8
-1 1210
-1 1211
-1 1212
-1 1213
12 376
81 40
82 6
83 53
84 5
86 26
88 5
93 9
-1 1214
-1 1215
-1 1216
-1 1217
12 377
81 41
82 7
83 54
84 6
86 27
88 6
93 10
-1 1218
-1 1219
-1 1220
-1 1221
12 378
81 42
82 8
83 55
84 7
86 28
88 7
93 11
-1 1222
-1 1223
-1 1224
-1 1225
12 379
81 43
82 9
83 56
84 8
86 29
88 8
93 12
-1 1226
-1 1227
-1 1228
-1 1229
12 380
81 44
82 10
83 57
84 9
86 30
88 9
93 13
-1 1230
-1 1231
-1 1232
-1 1233
12 381
82 11
83 58
84 10
86 31
88 10
93 14
-1 1234
-1 1235
-1 1236
12 382
82 12
83 59
84 11
86 32
88 11
93 15
-1 1237
-1 1238
-1 1239
12 383
82 13
83 60
84 12
86 33
87 1
88 12
89 1
93 16
-1 1240
-1 1241
-1 1242
12 384
82 14
83 61
84 13
86 34
87 2
88 13
89 2
93 17
-1 1243
-1 1244
-1 1245
12 385
82 15
83 62
84 14
86 35
87 3
88 14
89 3
91 1
93 18
-1 1246
-1 1247
-1 1248
12 386
82 16
83 63
84 15
85 1
86 36
87 4
88 15
89 4
91 2
93 19
-1 1249
-1 1250
-1 1251
12 387
83 64
84 16
85 2
86 37
87 5
88 16
89 

-1 1509
-1 1510
-1 1511
12 484
119 18
122 54
123 70
-1 1512
-1 1513
-1 1514
12 485
119 19
122 55
123 71
-1 1515
-1 1516
-1 1517
12 486
122 56
123 72
-1 1518
-1 1519
12 487
122 57
123 73
-1 1520
-1 1521
12 488
122 58
123 74
-1 1522
12 489
122 59
123 75
-1 1523
12 490
122 60
123 76
-1 1524
12 491
122 61
123 77
-1 1525
12 492
122 62
123 78
-1 1526
12 493
122 63
123 79
-1 1527
12 494
122 64
123 80
-1 1528
12 495
122 65
123 81
-1 1529
12 496
122 66
123 82
-1 1530
12 497
122 67
123 83
-1 1531
12 498
122 68
123 84
-1 1532
12 499
122 69
123 85
-1 1533
12 500
122 70
123 86
-1 1534
12 501
122 71
123 87
-1 1535
12 502
122 72
123 88
-1 1536
12 503
122 73
123 89
-1 1537
12 504
122 74
123 90
-1 1538
12 505
122 75
123 91
-1 1539
12 506
122 76
123 92
-1 1540
12 507
122 77
123 93
-1 1541
12 508
122 78
123 94
-1 1542
12 509
122 79
123 95
-1 1543
12 510
122 80
123 96
-1 1544
12 511
122 81
123 97
-1 1545
12 512
122 82
123 98
-1 1546
12 513
122 83
123 99
12 514
122 84
123 100
12 515
122 85
123 101
12 516
1

-1 2010
-1 2011
-1 2012
122 242
123 258
-1 2013
-1 2014
-1 2015
122 243
123 259
-1 2016
-1 2017
-1 2018
122 244
123 260
-1 2019
-1 2020
-1 2021
122 245
123 261
-1 2022
-1 2023
-1 2024
122 246
123 262
-1 2025
-1 2026
-1 2027
122 247
123 263
-1 2028
-1 2029
-1 2030
122 248
123 264
-1 2031
-1 2032
-1 2033
122 249
123 265
-1 2034
-1 2035
-1 2036
122 250
123 266
-1 2037
-1 2038
-1 2039
122 251
123 267
-1 2040
-1 2041
-1 2042
122 252
123 268
-1 2043
-1 2044
-1 2045
122 253
123 269
-1 2046
-1 2047
-1 2048
122 254
123 270
-1 2049
-1 2050
-1 2051
122 255
123 271


In [ ]:
cv2.imshow("window_name",frame[161:374 ,737:931 ]) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [164]:
np.array(img_folders, np.int32)

array([20, 12,  5, 14, 15,  2,  1,  6,  7, 19, 17, 11,  0, 13, 18,  8,  9,
        4,  3, 10, 16], dtype=int32)

In [67]:
np.unique(types)

array(['Car', 'Cyclist', 'DontCare', 'Misc', 'Pedestrian', 'Tram',
       'Truck', 'Van'], dtype='<U10')

In [140]:
a = {"1", "2"}

In [142]:
if(not "3" in a): print("yes")

yes


In [166]:
max_track

917